# Data Preparation

Prepare train/val/test splits for coconut tree detection.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-object-detection-on-aerial-imagery/blob/master/notebooks/experiment.ipynb)

In [1]:
# !pip install dl4cv_oda

In [2]:
import requests
import geopandas as gpd
from pathlib import Path
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)

In [3]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded OSM data")

if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Cleaned {count} trees")

if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Downloaded tiles")

In [4]:
if not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"Clipped {stats['total_trees']} trees to {stats['processed']} tiles")
    
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Converted to YOLO format")
    
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42)
    print(f"Split: train={train_count}, val={val_count}, test={test_count}")
    
    config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
    print(f"Config: {config_file}")

print("Data preparation complete")

Data preparation complete


In [5]:
from datetime import datetime
import pandas as pd
from ultralytics import YOLO, RTDETR
import torch
import yaml

## Configuration

In [6]:
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)


# for training
SEED = 64
IMG_SIZE = 256
EPOCHS = 20 
PATIENCE = 5
BATCH = 16

# for hyper param tuning
TUNE_MODELS = True
TUNE_ITERATIONS = 1
TUNE_EPOCHS = 10

MODELS = [
    {"name": "yolov8l", "weights": "yolov8l.pt"},
    {"name": "yolo12l", "weights": "yolo12l.pt"},
    {"name": "rtdetr-l", "weights": "rtdetr-l.pt"},
]

torch.manual_seed(SEED)
exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Experiment: {exp_id}")
print(f"Models: {[m['name'] for m in MODELS]}")

Experiment: 20260111_172004
Models: ['yolov8l', 'yolo12l', 'rtdetr-l']


## Hyperparameter Tuning & Training 

In [7]:

results = []

for model_cfg in MODELS:
    name = model_cfg['name']

    model = RTDETR(model_cfg['weights']) if 'rtdetr' in name.lower() else YOLO(model_cfg['weights'])
    
    
    if TUNE_MODELS:
        tuning_name = f"{exp_id}_{name}_tune"
            
        print(f"\nTuning {name} for {TUNE_ITERATIONS} iterations")
        model.tune(
            data=str(YOLO_DIR / "config.yaml"),
            name = tuning_name,
            epochs=TUNE_EPOCHS,
            iterations=TUNE_ITERATIONS,
            imgsz=IMG_SIZE,
            plots=False,
            save=False,
            val=True
        )

        print(f"{name} tuning complete. Best hyperparameters saved to runs/detect/{tuning_name}/best_hyperparameters.yaml")
    

    tuned_params = {}
    best_cfg_path = Path(f"runs/detect/{exp_id}_{name}_tune/best_hyperparameters.yaml")

    if best_cfg_path.exists():
        with open(best_cfg_path, 'r') as f:
            tuned_params = yaml.safe_load(f)
        # Fix the float error: force to int
        if 'close_mosaic' in tuned_params:
            tuned_params['close_mosaic'] = int(tuned_params['close_mosaic'])


    # train the model  

    model.train(
        data=str(YOLO_DIR / "config.yaml"),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        patience=PATIENCE,
        batch=BATCH,
        seed=SEED,
        name=f"{exp_id}_{name}",
        project="runs",
        plots=False,
        verbose=False,
        **tuned_params
    )
    
    val_metrics = model.val(split='val', verbose=False)
    test_metrics = model.val(split='test', verbose=False)
    
    val_p, val_r = float(val_metrics.box.mp), float(val_metrics.box.mr)
    test_p, test_r = float(test_metrics.box.mp), float(test_metrics.box.mr)
    
    val_f1 = 2 * (val_p * val_r) / (val_p + val_r + 1e-6)
    test_f1 = 2 * (test_p * test_r) / (test_p + test_r + 1e-6)
    
    results.append({
        'model': name,
        'tuned': TUNE_MODELS,
        'val_precision': val_p,
        'val_recall': val_r,
        'val_f1': val_f1,
        'val_map50': float(val_metrics.box.map50),
        'test_precision': test_p,
        'test_recall': test_r,
        'test_f1': test_f1,
        'test_map50': float(test_metrics.box.map50),
    })
    
    print(f"{name}: val_f1={val_f1:.4f}, test_f1={test_f1:.4f}, test_map50={results[-1]['test_map50']:.4f}")




Tuning yolov8l for 1 iterations
Tuner: Initialized Tuner instance with 'tune_dir=/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/20260111_172004_yolov8l_tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/1 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0, 'close_mosaic': 10}
New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, au

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/10      6.57G      1.549     0.6451     0.4969        259        256: 100% ━━━━━━━━━━━━ 20/20 3.7it/s 5.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.5it/s 1.2s3.4s
                   all         93       2846      0.073      0.408     0.0825     0.0163

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/10      7.36G      0.951     0.7102     0.2494        241        256: 0% ──────────── 0/20  0.2s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/10      7.36G     0.8864     0.6189     0.2013        374        256: 100% ━━━━━━━━━━━━ 20/20 6.5it/s 3.1s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.0it/s 0.2s.3s
                   all         93       2846      0.592      0.553       0.51      0.207

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       3/10      7.36G     0.8703     0.5989     0.1879        220        256: 0% ──────────── 0/20  0.2s

...

       6/20      8.42G     0.8723     0.4687     0.1599        529        256: 100% ━━━━━━━━━━━━ 20/20 6.0it/s 3.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.8it/s 0.2s.3s
                   all         93       2846      0.683      0.585      0.536      0.162

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       7/20      4.48G     0.8381      0.507     0.1762        402        256: 0% ──────────── 0/20  0.1s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       7/20       7.5G     0.8687     0.4768     0.1672        433        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.7it/s 0.2s.3s
                   all         93       2846      0.628      0.606       0.51       0.12

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       8/20      7.58G     0.8217     0.4794     0.1446        493        256: 5% ╸─────────── 1/20 1.8it/s 0.3s<10.3s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       8/20      7.58G     0.8314     0.4832     0.1436        532        256: 100% ━━━━━━━━━━━━ 20/20 6.0it/s 3.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.8it/s 0.2s.3s
                   all         93       2846      0.639      0.592      0.512      0.126
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 3, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

8 epochs completed in 0.010 hours.
Optimizer stripped from /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/20260111_172004_rtdetr-l/weights/last.pt, 66.2MB
Optimizer stripped from /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/20260111_172004_rtdetr-l/weights/best.pt, 66.2MB

Validating /home/krschap/academia/dl4cv-object

## Results

In [9]:
import json 

df = pd.DataFrame(results)

summary = {
    'exp_id': exp_id,
    'seed': SEED,
    'epochs': EPOCHS,
    'img_size': IMG_SIZE,
    'batch': BATCH,
    'patience': PATIENCE,
    'models': [m['name'] for m in MODELS],
    'results': results,
    'best_model': results[df['test_map50'].idxmax()]['model'] if len(results) > 0 else None,
    'best_test_map50': float(df['test_map50'].max()) if len(results) > 0 else 0.0
}

with open(RESULTS_DIR / f"{exp_id}.json", 'w') as f:
    json.dump(summary, f, indent=2)

print("\nResults")
print(df[['model', 'val_f1', 'val_map50', 'test_f1', 'test_map50']].to_string(index=False))
print(f"\nBest: {summary['best_model']} (test_map50={summary['best_test_map50']:.4f})")
print(f"Saved: results/{exp_id}.json")


Results
   model   val_f1  val_map50  test_f1  test_map50
 yolov8l 0.116242   0.045339 0.109363    0.041798
 yolo12l 0.310131   0.139054 0.329094    0.154523
rtdetr-l 0.630970   0.564333 0.657056    0.602917

Best: rtdetr-l (test_map50=0.6029)
Saved: results/20260111_172004.json


## Analysis

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for model_cfg in MODELS:
    name = model_cfg['name']
    csv_file = Path(f"runs/{exp_id}_{name}/results.csv")
    
    if csv_file.exists():
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.strip()
        
        p = df['metrics/precision(B)']
        r = df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        
        axes[0].plot(f1, label=name, linewidth=2)
        axes[1].plot(df['metrics/mAP50(B)'], label=name, linewidth=2)

axes[0].set_title('F1 Score')
axes[0].set_ylabel('F1')
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].set_title('mAP50')
axes[1].set_ylabel('mAP50')
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / f"curves_{exp_id}.png", dpi=150)
plt.show()